In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.read.csv("data/source.csv", header=True, inferSchema=True)
df.printSchema()
df.show()


root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [3]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("source_id", StringType()),
    StructField("source_username", StringType()),
])

df = spark.read.csv("data/source.csv", header=True, schema=schema)
df.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



In [4]:
#df.write.json("spark-json")

In [5]:
df = spark.read.csv('data/case.csv', header=True, inferSchema=True)
df.printSchema()
df.show(5, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  


In [6]:
df = df.withColumnRenamed("SLA_due_date", "case_due_date")

In [7]:
df.explain()

== Physical Plan ==
*(1) Project [case_id#62, case_opened_date#63, case_closed_date#64, SLA_due_date#65 AS case_due_date#161, case_late#66, num_days_late#67, case_closed#68, dept_division#69, service_request_type#70, SLA_days#71, case_status#72, source_id#73, request_address#74, council_district#75]
+- FileScan csv [case_id#62,case_opened_date#63,case_closed_date#64,SLA_due_date#65,case_late#66,num_days_late#67,case_closed#68,dept_division#69,service_request_type#70,SLA_days#71,case_status#72,source_id#73,request_address#74,council_district#75] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/cryptobandido/codeup-data-science/spark-exercises/data/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...




In [8]:
df.groupby('case_late', 'case_closed').count().show()

+---------+-----------+------+
|case_late|case_closed| count|
+---------+-----------+------+
|       NO|        YES|735616|
|      YES|        YES| 87978|
|       NO|         NO| 11585|
|      YES|         NO|  6525|
+---------+-----------+------+



In [9]:
df = df.withColumn("case_late", expr("case_late == 'YES'"))

In [10]:
df = df.withColumn("case_closed", expr("case_closed == 'YES'"))

In [11]:
df.printSchema()
df.show(5, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- case_due_date: string (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true               

In [12]:
df.select(col('council_district'), format_string("%03d", col("council_district"))).show()

+----------------+-------------------------------------+
|council_district|format_string(%03d, council_district)|
+----------------+-------------------------------------+
|               5|                                  005|
|               3|                                  003|
|               3|                                  003|
|               3|                                  003|
|               7|                                  007|
|               7|                                  007|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|                                  004|
|               4|             

In [13]:
df = df.withColumn("council_district", format_string("%03d", col("council_district")))

In [14]:
df.explain()


== Physical Plan ==
*(1) Project [case_id#62, case_opened_date#63, case_closed_date#64, SLA_due_date#65 AS case_due_date#161, (case_late#66 = YES) AS case_late#212, num_days_late#67, (case_closed#68 = YES) AS case_closed#227, dept_division#69, service_request_type#70, SLA_days#71, case_status#72, source_id#73, request_address#74, format_string(%03d, council_district#75) AS council_district#326]
+- FileScan csv [case_id#62,case_opened_date#63,case_closed_date#64,SLA_due_date#65,case_late#66,num_days_late#67,case_closed#68,dept_division#69,service_request_type#70,SLA_days#71,case_status#72,source_id#73,request_address#74,council_district#75] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/cryptobandido/codeup-data-science/spark-exercises/data/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...




In [15]:
df.show(5, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 case_due_date        | 9/26/20 0:42         
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 case_due_date        | 1/5/18 8:30          
 case_late            | false                
 num_days_late        | -2.0126041

In [16]:
fmt = "M/d/yy H:mm"

df.select(
    "case_opened_date",
    to_timestamp("case_opened_date", fmt)
).show(5)

+----------------+-----------------------------------------------+
|case_opened_date|to_timestamp(`case_opened_date`, 'M/d/yy H:mm')|
+----------------+-----------------------------------------------+
|     1/1/18 0:42|                            2018-01-01 00:42:00|
|     1/1/18 0:46|                            2018-01-01 00:46:00|
|     1/1/18 0:48|                            2018-01-01 00:48:00|
|     1/1/18 1:29|                            2018-01-01 01:29:00|
|     1/1/18 1:34|                            2018-01-01 01:34:00|
+----------------+-----------------------------------------------+
only showing top 5 rows



In [17]:
df = df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
df = df.withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
df = df.withColumn("case_due_date", to_timestamp("case_due_date", fmt))

In [18]:
df.printSchema()
df.show(3, vertical=True)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true      

In [19]:
df.explain()

== Physical Plan ==
*(1) Project [case_id#62, gettimestamp(case_opened_date#63, M/d/yy H:mm, Some(America/Chicago)) AS case_opened_date#425, gettimestamp(case_closed_date#64, M/d/yy H:mm, Some(America/Chicago)) AS case_closed_date#440, gettimestamp(SLA_due_date#65, M/d/yy H:mm, Some(America/Chicago)) AS case_due_date#455, (case_late#66 = YES) AS case_late#212, num_days_late#67, (case_closed#68 = YES) AS case_closed#227, dept_division#69, service_request_type#70, SLA_days#71, case_status#72, source_id#73, request_address#74, format_string(%03d, council_district#75) AS council_district#326]
+- FileScan csv [case_id#62,case_opened_date#63,case_closed_date#64,SLA_due_date#65,case_late#66,num_days_late#67,case_closed#68,dept_division#69,service_request_type#70,SLA_days#71,case_status#72,source_id#73,request_address#74,council_district#75] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/cryptobandido/codeup-data-science/spark-exercises/data/case.csv], Pa

In [20]:
df.select(
    col("request_address"),
    lower(trim("request_address"))
).show(truncate=False)

+----------------------------------------+----------------------------------------+
|request_address                         |lower(trim(request_address))            |
+----------------------------------------+----------------------------------------+
|2315  EL PASO ST, San Antonio, 78207    |2315  el paso st, san antonio, 78207    |
|2215  GOLIAD RD, San Antonio, 78223     |2215  goliad rd, san antonio, 78223     |
|102  PALFREY ST W, San Antonio, 78223   |102  palfrey st w, san antonio, 78223   |
|114  LA GARDE ST, San Antonio, 78223    |114  la garde st, san antonio, 78223    |
|734  CLEARVIEW DR, San Antonio, 78228   |734  clearview dr, san antonio, 78228   |
|BANDERA RD and BRESNAHAN                |bandera rd and bresnahan                |
|10133  FIGARO CANYON, San Antonio, 78251|10133  figaro canyon, san antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|10133  figaro canyon, san antonio, 78251|
|10133  FIGARO CANYON, San Antonio, 78251|10133  figaro canyon, san antonio,

In [21]:
df = df.withColumn("request_address", lower(trim("request_address")))

In [22]:
df.printSchema()
df.show(3, vertical=True, truncate=False)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- case_due_date: timestamp (nullable = true)
 |-- case_late: boolean (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: boolean (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = false)

-RECORD 0-----------------------------------------------------
 case_id              | 1014127332                            
 case_opened_date     | 2018-01-01 00:42:00                   
 case_closed_date     | 2018-01-01 12:29:00                   
 case_due_date        | 2020-09-26 00:42:00                   
 case_late            | false            

In [23]:
df.select(
    "request_address",
    regexp_extract("request_address", r"(\d+)$", 1)
).show(truncate=False)

+----------------------------------------+------------------------------------------+
|request_address                         |regexp_extract(request_address, (\d+)$, 1)|
+----------------------------------------+------------------------------------------+
|2315  el paso st, san antonio, 78207    |78207                                     |
|2215  goliad rd, san antonio, 78223     |78223                                     |
|102  palfrey st w, san antonio, 78223   |78223                                     |
|114  la garde st, san antonio, 78223    |78223                                     |
|734  clearview dr, san antonio, 78228   |78228                                     |
|bandera rd and bresnahan                |                                          |
|10133  figaro canyon, san antonio, 78251|78251                                     |
|10133  figaro canyon, san antonio, 78251|78251                                     |
|10133  figaro canyon, san antonio, 78251|78251       

In [24]:
df = df.withColumn("zipcode", regexp_extract("request_address", r"(\d+)$", 1))


In [25]:
df

DataFrame[case_id: int, case_opened_date: timestamp, case_closed_date: timestamp, case_due_date: timestamp, case_late: boolean, num_days_late: double, case_closed: boolean, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: string, zipcode: string]

In [26]:
df.show(5, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 005                  
 zipcode              | 78207                
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-03 08:11:00  
 case_due_date        | 2018-01-05 08:30:00  
 case_late            | false     

In [27]:
# case_lifetime
#   - if case is closed then diff between close and open dates -- days_to_close
#   - else diff between open date and now -- case_age

df.select(
    "case_opened_date",
    "case_closed_date",
    "case_closed",
    datediff(current_timestamp(), "case_opened_date").alias("case_age"),
    datediff("case_closed_date", "case_opened_date").alias("days_to_close"),
).withColumn(
    "case_lifetime",
    when(col("case_closed"), col("days_to_close")).otherwise(col("case_age")),
).show(10)


+-------------------+-------------------+-----------+--------+-------------+-------------+
|   case_opened_date|   case_closed_date|case_closed|case_age|days_to_close|case_lifetime|
+-------------------+-------------------+-----------+--------+-------------+-------------+
|2018-01-01 00:42:00|2018-01-01 12:29:00|       true|    1065|            0|            0|
|2018-01-01 00:46:00|2018-01-03 08:11:00|       true|    1065|            2|            2|
|2018-01-01 00:48:00|2018-01-02 07:57:00|       true|    1065|            1|            1|
|2018-01-01 01:29:00|2018-01-02 08:13:00|       true|    1065|            1|            1|
|2018-01-01 01:34:00|2018-01-01 13:29:00|       true|    1065|            0|            0|
|2018-01-01 06:28:00|2018-01-01 14:38:00|       true|    1065|            0|            0|
|2018-01-01 06:57:00|2018-01-02 15:32:00|       true|    1065|            1|            1|
|2018-01-01 06:58:00|2018-01-02 15:32:00|       true|    1065|            1|            1|

In [28]:
df = df.withColumn("case_age", datediff(current_timestamp(), "case_opened_date"))
df = df.withColumn("days_to_close", datediff("case_closed_date", "case_opened_date"))
df = df.withColumn("case_lifetime", when(col("case_closed"), col("days_to_close")).otherwise(col("case_age")))

In [29]:
depts = spark.read.csv('data/dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('data/source.csv', header=True, inferSchema=True)

df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')

In [30]:
df.show()

+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+--------------------+----------------+-------+--------+-------------+-------------+--------------------+----------------------+-------------------+---------------+
|source_id|   dept_division|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|     request_address|council_district|zipcode|case_age|days_to_close|case_lifetime|           dept_name|standardized_dept_name|dept_subject_to_SLA|source_username|
+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+--------------------+----------------+-------+--------+-------------+-------------+----------

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [32]:
# this is so we can query our dataframe with spark sql
df.createOrReplaceTempView('df')

In [33]:
spark.sql('''
SELECT DATEDIFF(current_timestamp, case_due_date) AS days_past_due
FROM df
WHERE NOT case_closed
ORDER BY days_past_due DESC
LIMIT 15
''').show()

+-------------+
|days_past_due|
+-------------+
|         1414|
|         1414|
|         1414|
|         1413|
|         1411|
|         1407|
|         1407|
|         1406|
|         1405|
|         1405|
|         1401|
|         1400|
|         1400|
|         1400|
|         1397|
+-------------+



### 2. How many Stray Animal cases are there?

In [34]:
df.filter(df.service_request_type == 'Stray Animal').count()

27361

In [35]:
(
    df.groupBy('service_request_type')
    .count()
    .filter(expr('service_request_type == "Stray Animal"'))
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|27361|
+--------------------+-----+



### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [36]:
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

In [37]:
# instructors way

(
    df.filter(expr("dept_division == 'Field Operations'"))
    .filter(expr('service_request_type != "Officer Standby"'))
    .count()
)

116295

### 4. Convert the council_district column to a string column.

In [ ]:
# Already did this in the prep above from the lesson

### 5. Extract the year from the case_closed_date column.

In [38]:
df.select('case_closed_date', year('case_closed_date')).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [39]:
(
    df.withColumn('num_hours_late', df.num_days_late * 24)
    .select('num_days_late', 'num_hours_late')
    .show()
)

+-------------------+-------------------+
|      num_days_late|     num_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
|       -29.74398148| -713.8555555199999|
|       -14.70673611|      -352.96166664|
|       -14.70662037|      -352.95888888|
|       -14.70662037|      -352.95888888|
|       -14.70649306|      -352.95583344|
|       -14.70649306|      -352.95583344|
|       -14.70636574|      -352.95277776|
|          -14.70625|-352.95000000000005|
|       -14.70636574|      -352.95277776|
|       -14.70623843|-352.94972232000003|
|-14.705891199999998|-352.94138879999997|
|       -14.70600694|      -352.94416656|
|       -14.70576389|      -352.93833336|
|       -14.70576389|      -352.93833336|
|       -14.70564815|       -352.9355556|
+-------------------+-------------

### 7. Join the case data with the source and department data.

In [40]:
# already done in the prep above

sources.show()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows



### 8. Are there any cases that do not have a request source?

In [41]:
(
    df.select(df.source_id.isNull().cast('int').alias('is_null'))
    .agg(sum('is_null'))
    .show()
)

+------------+
|sum(is_null)|
+------------+
|           0|
+------------+



In [42]:
df.filter(col('source_id').isNull()).show(vertical=True)

(0 rows)



### 9. What are the top 10 service request types in terms of number of requests?

In [43]:
(
    df.groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?

In [44]:
(
    df.where('case_late') # just the rows where case_late == true
    .groupBy('service_request_type')
    .agg(mean('num_days_late').alias('n_days_late'), count('*').alias('n_cases'))
    .sort(desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.2051760849428 |262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942615|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [45]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

+-------------------------+---------+------------+
|dept_name                |days_late|n_cases_late|
+-------------------------+---------+------------+
|Metro Health             |6.5      |854         |
|Solid Waste Management   |7.1      |33729       |
|Trans & Cap Improvements |10.7     |5529        |
|Parks and Recreation     |22.4     |3810        |
|Animal Care Services     |23.4     |23751       |
|Code Enforcement Services|48.1     |25467       |
|Development Services     |67.2     |840         |
|Customer Service         |88.2     |2035        |
|null                     |210.9    |132         |
+-------------------------+---------+------------+



In [46]:
df.groupby('dept_name').count().show(truncate=False)

+-------------------------+------+
|dept_name                |count |
+-------------------------+------+
|Animal Care Services     |119362|
|null                     |198   |
|Solid Waste Management   |286287|
|Development Services     |1397  |
|Trans & Cap Improvements |97841 |
|Customer Service         |2889  |
|Metro Health             |5313  |
|Parks and Recreation     |19964 |
|Code Enforcement Services|321984|
|City Council             |34    |
+-------------------------+------+



### 12. How do number of days late depend on department and request type?

In [47]:
(
    df.filter("case_closed")
     .filter("case_late")
    .groupby("standardized_dept_name", "service_request_type")
    .agg(avg("num_days_late").alias("days_late"), count("*").alias("n_cases"))
    .withColumn("days_late", round(col("days_late"), 1))
    .sort(asc("days_late"))
    .show(40, truncate=False)
)

+------------------------+-----------------------------------------------+---------+-------+
|standardized_dept_name  |service_request_type                           |days_late|n_cases|
+------------------------+-----------------------------------------------+---------+-------+
|Animal Care Services    |Injured Animal(Critical)                       |0.1      |578    |
|Animal Care Services    |Officer Standby                                |0.1      |97     |
|Trans & Cap Improvements|Sign Installations SMO (NEW)                   |0.1      |1      |
|Trans & Cap Improvements|Tree Fallen                                    |0.1      |8      |
|Trans & Cap Improvements|Herbicide - Grass in curb or street            |0.1      |1      |
|Solid Waste             |Additional Organics Cart Request               |0.2      |3      |
|Solid Waste             |Brush No Notice                                |0.2      |2      |
|Animal Care Services    |Animal Cruelty(Critical)                    

In [48]:
(
    df.filter(col("service_request_type") == "Stray Animal")
    .select("case_opened_date", "case_due_date", "case_closed_date")
    .withColumn("days_to_resolve", datediff("case_due_date", "case_opened_date"))
    .show()
)

+-------------------+-------------------+-------------------+---------------+
|   case_opened_date|      case_due_date|   case_closed_date|days_to_resolve|
+-------------------+-------------------+-------------------+---------------+
|2018-01-01 00:42:00|2020-09-26 00:42:00|2018-01-01 12:29:00|            999|
|2018-01-01 08:39:00|2020-09-26 08:39:00|2018-01-01 12:30:00|            999|
|2018-01-01 09:51:00|2020-09-26 09:51:00|2018-01-01 11:53:00|            999|
|2018-01-01 10:39:00|2020-09-26 10:39:00|2018-01-01 12:29:00|            999|
|2018-01-01 10:44:00|2020-09-26 10:44:00|2018-01-01 12:43:00|            999|
|2018-01-01 10:52:00|2020-09-26 10:52:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:11:00|2020-09-26 11:11:00|2018-01-01 11:53:00|            999|
|2018-01-01 11:46:00|2020-09-26 11:46:00|2018-01-01 12:26:00|            999|
|2018-01-01 11:58:00|2020-09-26 11:58:00|2018-01-01 12:26:00|            999|
|2018-01-01 12:28:00|2020-09-26 12:28:00|2018-01-01 12:44:00|   